In [1]:
import os
import numpy as np
import glob
import pandas as pd
from pyproj import Transformer

In [2]:
# Podatki o nadmorski višini (digitalni model višin) so na voljo na https://ipi.eprostor.gov.si/jgp/data
def read_dmv_data_files(root):
    data = []
    for file_path in glob.iglob(f"{root}/**/*.xyz", recursive=True):
        with open(file_path, "r") as f:
            file_data = []
            for row in f:
                file_data.append([float(el) for el in row.split(" ")])
            data.append(np.array(file_data))

        print(f"{file_path} processed.")
    
    return np.concatenate(data, axis=0)

In [52]:
def transform_coords(data):
    df = pd.DataFrame(data, columns=["x", "y", "z"])
    transformer = Transformer.from_crs("EPSG:3794", "epsg:4326")
    
    # Počasno na večjih količinah podatkov
    df["Lat"] = df.apply(lambda row: transformer.transform(getattr(row, "x"), getattr(row, "y"))[0], axis=1)
    df["Lon"] = df.apply(lambda row: transformer.transform(getattr(row, "x"), getattr(row, "y"))[1], axis=1)
    
    return df
    

def bbox_slice(df, bbox):
    np.save(
        "data.npy",
        df.loc[
            (df.Lat >= bbox[0]) & 
            (df.Lat <= bbox[1]) & 
            (df.Lon >= bbox[2]) & 
            (df.Lon <= bbox[3])
        ][["x", "y", "z"]].to_numpy()
    )
    
    return np.load("data.npy")

In [59]:
data = read_dmv_data_files("data/DMV0050_SZ/C08/")
print("________________")
print(data)

data/DMV0050_SZ/C08\VTC0801.xyz processed.
data/DMV0050_SZ/C08\VTC0802.xyz processed.
data/DMV0050_SZ/C08\VTC0803.xyz processed.
data/DMV0050_SZ/C08\VTC0804.xyz processed.
data/DMV0050_SZ/C08\VTC0805.xyz processed.
data/DMV0050_SZ/C08\VTC0806.xyz processed.
data/DMV0050_SZ/C08\VTC0807.xyz processed.
data/DMV0050_SZ/C08\VTC0808.xyz processed.
data/DMV0050_SZ/C08\VTC0809.xyz processed.
data/DMV0050_SZ/C08\VTC0810.xyz processed.
data/DMV0050_SZ/C08\VTC0811.xyz processed.
data/DMV0050_SZ/C08\VTC0812.xyz processed.
data/DMV0050_SZ/C08\VTC0813.xyz processed.
data/DMV0050_SZ/C08\VTC0814.xyz processed.
data/DMV0050_SZ/C08\VTC0815.xyz processed.
data/DMV0050_SZ/C08\VTC0816.xyz processed.
data/DMV0050_SZ/C08\VTC0817.xyz processed.
data/DMV0050_SZ/C08\VTC0818.xyz processed.
data/DMV0050_SZ/C08\VTC0819.xyz processed.
data/DMV0050_SZ/C08\VTC0820.xyz processed.
data/DMV0050_SZ/C08\VTC0821.xyz processed.
data/DMV0050_SZ/C08\VTC0822.xyz processed.
data/DMV0050_SZ/C08\VTC0823.xyz processed.
data/DMV005

In [60]:
df = transform_coords(data)

In [61]:
df.head()

,x,y,z,Lat,Lon
0,410000.0,142000.0,1553.5,46.411696,13.829327
1,410005.0,142000.0,1550.1,46.411697,13.829392
2,410010.0,142000.0,1546.6,46.411698,13.829457
3,410015.0,142000.0,1543.2,46.411698,13.829522
4,410020.0,142000.0,1539.8,46.411699,13.829587


In [62]:
df["z"].max()

2864.7

In [63]:
# Robne kordinate lahko preprosto dobiš na OpenStreetMaps -> izvozi
data = bbox_slice(df, bbox=(46.3746,46.3812,13.8365,13.8538))

In [64]:
data

array([[411775. , 137850. ,   2288.1],
       [411780. , 137850. ,   2285. ],
       [411785. , 137850. ,   2282.4],
       ...,
       [410645. , 138600. ,   2481.8],
       [410650. , 138600. ,   2478.4],
       [410655. , 138600. ,   2475. ]])